In [ ]:
%pip install langgraph

In [20]:
from langgraph.graph import StateGraph, END
from langgraph.types import Command, interrupt
from typing import TypedDict
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    text: str

graph = StateGraph(State)

def node_a(state: State):
    print("Node A")
    return Command(
        goto="node_b",
        update={
            "text": state["text"] + "a"
        }
    )

def node_b(state: State):
    print("Node B")
    
    decision = interrupt("Where you want to go c or d type c/d")
    if decision == "c":
        return Command(
            goto="node_c",
            update={
                "text": state["text"] + "b"
            }
        )
    else:
        return Command(
            goto="node_d",
            update={
                "text": state["text"] + "b"
            }
        )

def node_c(state: State):
    print("Node C")
    return Command(
        goto=END,
        update={
            "text": state["text"] + "c"
        }
    )

def node_d(state: State):
    print("Node D")
    return Command(
        goto=END,
        update={
            "text": state["text"] + "d"
        }
    )

graph.add_node("node_a",node_a)
graph.add_node("node_b",node_b)
graph.add_node("node_c",node_c)
graph.add_node("node_d",node_d)

graph.set_entry_point("node_a")

app = graph.compile(checkpointer=memory)

config={
    "configurable": {
        "thread_id": 1
    }
}

first_res = app.invoke({
    "text": ""
}, config, stream_mode="updates")

first_res

Node A
Node B


[{'node_a': {'text': 'a'}},
 {'__interrupt__': (Interrupt(value='Where you want to go c or d type c/d', id='9aede5edb9495ac14c8f90425d59a5db'),)}]

In [6]:
print(app.get_state(config).next)

('node_b',)


In [21]:
second_res = app.invoke(Command(resume="c"), config, stream_mode="updates")

second_res

Node B
Node C


[{'node_b': {'text': 'ab'}}, {'node_c': {'text': 'abc'}}]